# Modeling And Simulating of Dynamical Systems: Biped Walker

## Rewrite everything in casadi library: Do not use nor sympy nor numpy
### Import libraries

In [2]:
import sympy as sp
import numpy as np
from sympy.physics.mechanics import *
from numpy import sin, cos, pi
from numpy.linalg import inv
from scipy.integrate import odeint, solve_bvp, solve_ivp
from scipy.interpolate import interp1d
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint, Bounds
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from sympy import atan, symbols, Matrix, Eq, solve
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                     Particle, Point)
import sympy.physics.mechanics.system as system
from sympy.physics.vector import init_vprinting
import math
import casadi as ca
from casadi import *
from casadi.tools import *

# Bestekoak be asko redundante ba badaeszpada
# imports needed for the code (run this block)
import numpy as np
import matplotlib
from numpy import sin, cos, pi
from numpy.linalg import inv
from scipy.integrate import odeint, solve_bvp
from scipy.interpolate import interp1d
from scipy.linalg import inv, solve_continuous_are, norm
from matplotlib import pyplot as plt
import matplotlib.pyplot as pltpiip
from matplotlib import animation
from matplotlib.patches import Rectangle
import sympy as sp
from sympy.physics.mechanics import *
from numpy.linalg import inv
from scipy.integrate import odeint, solve_bvp
from scipy.interpolate import interp1d 
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint, Bounds
import matplotlib
from sympy import atan, symbols, Matrix
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                     Particle, Point)
import sympy.physics.mechanics.system as system
from sympy.physics.vector import init_vprinting
# matplotlib qt
# from animate_pendulum import animate_pendulum
# from animate_walker import animate_walker
def to_2d(x): return x.reshape(-1, 1)
def to_1d(x): return x[:, 0]

### Given data

In [3]:
## gravity and time
g = 9.81
m_t = ca.MX.sym('m_t')
m_f = ca.MX.sym('m_f')
l_t = ca.MX.sym('l_t')
l_f = ca.MX.sym('l_f')

# Declare masses in kg
m1 = 6.8
m2 = 6.8
m3 = 3.2
m4 = 3.2
m5 = 20

mass = [m1, m2, m3, m4, m5]

## Declare lengths in m
l1 = 0.4 
l2 = 0.4
l3 = 0.4
l4 = 0.4
l5 = 0.625

length = [l1, l2, l3, l4, l5]

## Declare distance to gravity center in m
l_r1 = (0.163/0.4)*l1
l_r2 = (0.163/0.4)*l2
l_r3 = (0.128/0.4)*l3
l_r4 = (0.128/0.4)*l4
l_r5 = 0.2

l_r = [l_r1, l_r2, l_r3, l_r4, l_r5]

## Declare inertias in kg*m^2
I1 = (1/12)*(m1)*(l1**2)
I2 = (1/12)*(m2)*(l2**2)
I3 = (1/12)*(m3)*(l3**2)
I4 = (1/12)*(m2)*(l4**2)
I5 = 2.22

inertia = [I1, I2, I3, I4, I5]

### Angle + Input definition
- Calculus of the angles: q are the relative angles and theta, the absolute ones.
- Input: each part of the body is applied an independent torque, simulating a motor.


In [4]:
# Generate q and input matrix (u) in function of time
# Declare symbolic variables q1, q2, q3, q4, q5, dq1, dq2, dq3, dq4, dq5
q = ca.MX.sym('q', 5, 1)
dq = ca.MX.sym('dq', 5, 1)
u = ca.MX.sym('u', 5, 1)

# Calculate the angle of each part of the body in the absolute frame
def abs_ang_rad(q):
    return ca.horzcat(
        -q[4] - q[0] + pi,
        -q[4] - q[1] ,
        -q[4] - q[0] + q[2] + pi,
        -q[4] - q[1] + q[3] ,
        -q[4],
    )

abs_ang = abs_ang_rad(q)
print(abs_ang)
print(u)

horzcat((3.14159+((-q[4])-q[0])), ((-q[4])-q[1]), (3.14159+(((-q[4])-q[0])+q[2])), (((-q[4])-q[1])+q[3]), (-q[4]))
u


## Kinematics
### Position calculation
First the relative position of each part of the body is calculated and afterwards the absolute position is calculated by defining a chain of the body parts. All calculations are done in symbolic expressions.

In [5]:
def rel_pos(angles):

    ## Declare lengths in m
    l1 = 0.4
    l2 = 0.4
    l3 = 0.4
    l4 = 0.4
    l5 = 0.625

    length = [l1, l2, l3, l4, l5]

    ## Declare distance to gravity center in m
    l_r1 = 0.163
    l_r2 = 0.163
    l_r3 = 0.128
    l_r4 = 0.128
    l_r5 = 0.2

    l_r = [l_r1, l_r2, l_r3, l_r4, l_r5]

    # Calculate the absolute angles of each part of the body and print to verify
    theta = abs_ang_rad(angles)

    r_x = 0
    r_y = 0
    joint_x = 0
    joint_y = 0

    # Loop to calculate each relative position
    for i in range(5):
        # Relative position of gravity centres
        r_x_new = (length[i] - l_r[i]) * ca.sin(theta[i])
        r_y_new = (length[i] - l_r[i]) * ca.cos(theta[i])

        #Relative position of joints
        joint_x_new = length[i] * ca.sin(theta[i])
        joint_y_new = length[i] * ca.cos(theta[i])
        
        r_x = ca.horzcat(r_x, r_x_new)
        r_y = ca.horzcat(r_y, r_y_new)
        joint_x = ca.horzcat(joint_x, joint_x_new)
        joint_y = ca.horzcat(joint_y, joint_y_new)
    
    r = [r_x[1:], r_y[1:]]
    joint = [joint_x[1:], joint_y[1:]]

    # Return matrices for positions of gravity centres and joints
    return r, joint


#Calculate absolute positions of each element of the body
def abs_pos(angles):
    r, joint = rel_pos(angles)

    r_x = r[0]
    r_y = r[1]
    joint_x = joint[0]
    joint_y = joint[1]

    # Absolute position of Knee 1
    r_x_2 = r_x[2]
    r_y_2 = r_y[2]
    joint_x_2 = joint_x[2]
    joint_y_2 = joint_y[2]


    # Absolute position of Hip
    r_x_0 = r_x[0] + joint_x[2]
    r_y_0 = r_y[0] + joint_y[2]
    joint_x_0 = joint_x[0] + joint_x[2]
    joint_y_0 = joint_y[0] + joint_y[2]

    # Absolute position of Head
    r_x_4 = r_x[4] + joint_x_0
    r_y_4 = r_y[4] + joint_y_0
    joint_x_4 = joint_x[4] + joint_x_0
    joint_y_4 = joint_y[4] + joint_y_0

    # Absolute position of Knee 2
    r_x_1 = r_x[1] + joint_x_0
    r_y_1 = r_y[1] + joint_y_0
    joint_x_1 = joint_x[1] + joint_x_0
    joint_y_1 = joint_y[1] + joint_y_0

    # Absolute position of Foot 2
    r_x_3 = r_x[3] + joint_x_1
    r_y_3 = r_y[3] + joint_y_1
    joint_x_3 = joint_x[3] + joint_x_1
    joint_y_3 = joint_y[3] + joint_y_1

    r_x = ca.horzcat(r_x_0, r_x_1, r_x_2, r_x_3, r_x_4)
    r_y = ca.horzcat(r_y_0, r_y_1, r_y_2, r_y_3, r_y_4)
    r = [r_x, r_y]

    joint_x = ca.horzcat(joint_x_0, joint_x_1, joint_x_2, joint_x_3, joint_x_4)
    joint_y = ca.horzcat(joint_y_0, joint_y_1, joint_y_2, joint_y_3, joint_y_4)
    joint = [joint_x, joint_y]


    return r, joint

r, joint = abs_pos(q)

print("Absolute position of gravity centres:\n" 
       f"{r} \n\n" 
       "Absolute position of joints:\n"
       f"{joint} \n\n")

Absolute position of gravity centres:
[MX(@1=(3.14159+((-q[4])-q[0])), @2=(3.14159+(((-q[4])-q[0])+q[2])), @3=(0.4*sin(@2)), @4=((-q[4])-q[1]), @5=((0.4*sin(@1))+@3), horzcat(((0.237*sin(@1))+@3), ((0.237*sin(@4))+@5), (0.272*sin(@2)), ((0.272*sin((((-q[4])-q[1])+q[3])))+((0.4*sin(@4))+@5)), ((0.425*sin((-q[4])))+@5))), MX(@1=(3.14159+((-q[4])-q[0])), @2=(3.14159+(((-q[4])-q[0])+q[2])), @3=(0.4*cos(@2)), @4=((-q[4])-q[1]), @5=((0.4*cos(@1))+@3), horzcat(((0.237*cos(@1))+@3), ((0.237*cos(@4))+@5), (0.272*cos(@2)), ((0.272*cos((((-q[4])-q[1])+q[3])))+((0.4*cos(@4))+@5)), ((0.425*cos(q[4]))+@5)))] 

Absolute position of joints:
[MX(@1=(0.4*sin((3.14159+(((-q[4])-q[0])+q[2])))), @2=((0.4*sin((3.14159+((-q[4])-q[0]))))+@1), @3=((0.4*sin(((-q[4])-q[1])))+@2), horzcat(@2, @3, @1, ((0.4*sin((((-q[4])-q[1])+q[3])))+@3), ((0.625*sin((-q[4])))+@2))), MX(@1=(0.4*cos((3.14159+(((-q[4])-q[0])+q[2])))), @2=((0.4*cos((3.14159+((-q[4])-q[0]))))+@1), @3=((0.4*cos(((-q[4])-q[1])))+@2), horzcat(@2, @3, @1

### Function to calculate numerically the position of the body

In [24]:
# Calculus of simulated instance position
def sim_pos(q_simulated):

    # Simulate numerically the position of each joint
    r_sim, joint_sim = abs_pos(q_simulated)

    # Order joints to follow body structure
    joint_x_sym = joint_sim[0]
    joint_y_sym = joint_sim[1]
    pos_x = [0, joint_x_sym[2], joint_x_sym[0], joint_x_sym[4], joint_x_sym[0], joint_x_sym[1], joint_x_sym[3]]
    pos_y = [0, joint_y_sym[2], joint_y_sym[0], joint_y_sym[4], joint_y_sym[0], joint_y_sym[1], joint_y_sym[3]]

    return pos_x, pos_y

### Plotting function of the body

In [69]:
# PLOT
def plot_pos(pos_y, pos_x):
    plt.plot(pos_x, pos_y, marker='o', linestyle='--', color='b')  
    plt.title('Joint Plot')
    plt.ylabel('Y-axis')
    plt.xlabel('X-axis')
    plt.axis('equal')
    plt.grid(True)
    plt.show()

### Check of kinematics

In [8]:
q_0 = np.array([np.pi*5/6, -np.pi*4/6, 0, 0, 0])
pos_x_1, pos_y_1 = sim_pos(q_0)
print('X: ' + f'{pos_x_1}')
print('Y: ' + f'{pos_y_1}')

X: [0, DM(0.2), DM(0.4), DM(0.4), DM(0.4), DM(0.74641), DM(1.09282)]
Y: [0, DM(0.34641), DM(0.69282), DM(1.31782), DM(0.69282), DM(0.49282), DM(0.29282)]


## Dynamics
### Mass matrix

In [9]:
# Mass matrix
def M(q, m_f, m_t, l_f, l_t):

    #Define each element of the C
    M11 = (-1.4343*(l_f**2)*m_f) - ((l_f**2)*m_t) - (20*(l_f**2)) - (3.185 * l_f * m_f * ca.cos(q[2])) - (2 * l_f * m_t * ca.cos(q[2])) - (40 * l_t * m_t *  ca.cos(q[2])) - (2 * (l_t**2)*m_f) - (1.545 * (l_t**2) * m_t) - (20 * (l_t**2))
    M12 = 0.5925 * l_f * m_f * (0) * l_f * ca.sin(q[0] - q[1]) + (0 * l_t * ca.sin(-q[0]+q[1]+q[2])) - m_t * ((l_f * ca.sin(q[1] - q[3] + q[4]))*(l_f * ca.sin(q[0] + q[4] - pi) + l_t * ca.sin(q[0] - q[2] + q[4] - pi)) + (l_f * ca.cos(q[1] + q[4]) + 0.68 * l_t * ca.cos(q[1] - q[3] + q[4]))*(l_f * ca.cos(q[0] + q[4] - pi) + l_t * ca.cos(q[0] - q[2] + q[4] - pi)))
    M13 = l_t * (1.5925 * l_f * m_f * ca.cos(q[2]) + l_f * m_f * ca.cos(q[2]) + 20 * l_f * ca.cos(q[2]) + 2*l_t*m_f + 1.5457*l_t*m_t + 20*l_t)
    M14 = 0.68*l_t*m_t*((l_f*ca.sin(q[0] + q[4] - pi) + l_t*ca.sin(q[0] - q[2] + q[4] - pi))*ca.sin(q[1] - q[3] + q[4]) + (l_f*ca.cos(q[0] + q[4] - pi) + l_t*ca.cos(q[0] - q[2] + q[4] - pi)) * ca.cos(q[1] - q[2] + q[4]))
    M15 = -0.0833 * (l_f**2)*m_f - 0.545*(l_t**2)*m_t - 0.5*m_f*(0.702*(l_f**2) + 2.37*l_f*l_t*ca.cos(q[2]) + 2*(l_t**2)) - m_f*(0*(l_f**2)*ca.sin(q[0] - q[1]) - 0.5925*(l_f**2)*ca.cos(q[0] - q[1]) + (l_f**2) - 0*l_f*l_t*ca.sin(-q[0] + q[1] + q[2]) - 0.5925*l_f*l_t*ca.cos(-q[0] + q[1] + q[2]) + 2*l_f*l_t*ca.cos(q[2]) + l_t**2) - m_t*((l_f * ca.sin(q[0] + q[4] - pi) + l_t*ca.sin(q[0] - q[2] + q[4] - pi))*(l_f*ca.sin(q[1] + q[4])) + l_f*ca.sin(q[0] + q[4] - pi) + 0.68*l_t*ca.sin(q[1] - q[3] + q[4]) + l_t*ca.sin(q[0] - q[2] + q[4] - pi)) + (l_f*ca.cos(q[0] + q[4] - pi) + l_t*ca.cos(q[0] - q[2] + q[4] - pi))*(l_f*ca.cos(q[1] + q[4]) + l_f*ca.cos(q[0] + q[4] - pi) + 0.68*l_t*ca.cos(q[1] - q[3] + q[4]) + l_t*ca.cos(q[0] - q[2] + q[4] - pi)) - 20*(l_f*ca.sin(q[0] + q[4] - pi) + l_t*ca.sin(q[0] - q[2] + q[4] - pi))*(l_f*ca.sin(q[0] + q[4] - pi) + l_t*ca.sin(q[0] - q[2] +q[4] - pi) + 0.425*ca.sin(q[4])) - 20*(l_f*ca.cos(q[0] + q[4] - pi) + l_t*ca.cos(q[0] - q[2] + q[4] - pi))*(l_f*ca.cos(q[0] - q[2] + q[4]) + 0.425*ca.cos(q[4]))
    
    M21 = 0.5925*l_f*m_f*(0*l_f*ca.sin(q[0] - q[1]) + 0*l_f*ca.sin(-q[0] + q[1] + q[2]) + l_t*ca.cos(-q[0] + q[1] + q[2])) - 0.5*m_f*((2*l_f*ca.sin(q[1]+q[4]) + 1.36*l_t*ca.sin(q[1]-q[3]+q[4]))*(l_f*ca.sin(q[0]+q[4]-pi) + l_t*ca.sin(q[0]-q[2]+q[4]-pi)) + (2*l_f*ca.cos(q[1]+q[4]) + 1.36*l_t*ca.cos(q[1] - q[3] + q[4]))*(l_f*ca.cos(q[0]+q[4]-pi) + l_t*ca.cos(q[0] - q[2] + q[4] - pi)))
    M22 = -0.4343*(l_f**2)*m_f - (l_f**2)*m_t - 1.36*l_f*l_t*m_t*ca.cos(q[3]) - 0.0833*(l_t**2)*m_f - 0.4624*(l_t**2)*m_t
    M23 = l_t*(-0.5925*l_f*m_f*(0*ca.sin(-q[0]+q[1]+q[2]) + ca.cos(-q[0]+q[1]+q[2])) + 0.5*m_t*((2*l_f*sin(q[1] + q[4]) + 1.36*l_t*ca.sin(q[1]-q[3]+q[4]))*ca.sin(q[0]-q[2]+q[4]-pi) + (2*l_f*ca.cos(q[1]+q[4]) + 1.36*l_f*ca.cos(q[1]-q[3]+q[4]))*ca.cos(q[0]-q[2]+q[4]-pi)))
    M24 = l_t*(l_f*m_t**ca.cos(q[3]) + 0.0833*l_t*m_f + 0.4624*l_t*m_t)
    M25 = -0.0833*(l_f**2)*m_f + 0.5925*l_f*m_f*(0*l_f*ca.sin(q[0]-q[1]) + l_f*ca.cos(q[0] - q[1]) - 0.5925*l_f + 0*l_t*ca.sin(-q[0]+q[1]+q[2]) + l_t*ca.cos(-q[0]+q[1]+q[2])) - 0.0833*(l_t**2)*m_f - 0.5*((2*l_f*ca.sin(q[1]+q[4]) + 1.36*l_t*ca.sin(q[1]-q[3]+q[4]))*(l_f*ca.sin(q[1]+q[4]) + l_f*ca.sin(q[0]+q[4]-pi) + 0.68*l_t*ca.sin(q[1]-q[3]+q[4]) + l_t*ca.sin(q[0]-q[2]+q[4]-pi)) + (2*l_f*ca.cos(q[1]+q[4]) + 1.36*l_t*ca.cos(q[1]-q[3]+q[4]))*(l_f*ca.cos(q[1]+q[4])+l_f*ca.cos(q[0]+q[4]-pi) + 0.68*l_t*ca.cos(q[1]-q[3]+q[4]) + l_t*ca.cos(q[0]-q[2]+q[4]-pi)))

    M31 = l_t*(1.5925*l_f*m_f*ca.cos(q[2]) + l_f*m_t*ca.cos(q[2]) + 20*l_f*ca.cos(q[2]) + 2*l_t*m_f + 1.545*l_t*m_t + 20*l_t)
    M32 = l_t*(-0.5925*l_f*m_f*(0*ca.sin(-q[0]+q[1]+q[2]) + ca.cos(-q[0]+q[1]+q[2])) + m_t*(l_f*ca.sin(q[1]+q[4]) + 0.68*l_t*ca.sin(q[1]-q[3]+q[4]))*ca.sin(q[0]-q[2]+q[4]-pi) + (l_f*ca.cos(q[1]+q[4]) + 0.68*l_t*ca.cos(q[1]-q[3]+q[4]))*ca.cos(q[0]-q[2]+q[4]-pi))
    M33 = -(l_t**2)*(2*m_f + 1.5457*m_t +20)
    M34 = -0.68*(l_t**2)*m_t*(0*ca.sin(q[0]-q[1]-q[2]+q[3]) - ca.cos(q[0]-q[1]-q[2]+q[3]))
    M35 = l_t*(0*l_f*m_f*ca.sin(-q[0]+q[1]+q[2]) - 0.5925*l_f*m_f*ca.cos(-q[0]+q[1]+q[2]) + 1.5925*l_f*m_f*cos(q[3]) - 0*l_f*m_t*ca.sin(-q[0]+q[1]+q[2]) - l_f*m_t*ca.cos(-q[0]+q[1]+q[3]) + l_f*m_t*ca.cos(q[2]) + 20*l_f*ca.cos(q[2]) + 2*l_t*m_f - 0.68*l_t*m_t*ca.sin(q[0]-q[2])*ca.sin(q[1])*ca.cos(q[3]) - 0.68*l_t*m_t*ca.sin(q[0]+q[3])*ca.sin(q[2])*ca.cos(q[1]) + 0*l_t*m_t*ca.sin(q[0]-q[1]-q[2]+q[3]) + 0.68*l_t*m_t*ca.sin(q[0])*ca.sin(q[3])*ca.cos(q[1]+q[2]) - 0.68*l_t*m_t*ca.cos(q[1] - q[3])*ca.cos(q[0])*ca.cos(q[2]) + 1.545*l_t*m_t + 20*l_t + 0*ca.sin(q[0]-q[2]) - 8.5*ca.cos(q[0]-q[2]))

    M41 = 0.68*l_t*m_t*((l_f*ca.sin(q[0]+q[4]-pi) + l_t*ca.sin(q[0]-q[2]+q[4]-pi))*ca.sin(q[1]-q[3]+q[4]) + (l_f*ca.cos(q[0]+q[4]-pi) + l_f*ca.cos(q[0]-q[2]+q[4]-pi))*ca.cos(q[1]-q[3]+q[4]))
    M42 = l_t*(0.68*m_f*m_t*ca.cos(q[3]) + 0.0833*l_f*m_f + 0.4624*l_t*m_t)
    M43 = -0.68*(l_t**2)*m_t*(0*ca.sin(q[0]-q[1]-q[2]+q[3]) - ca.cos(q[0]-q[1]-q[2]+q[3]))
    M44 = (l_t**2)*(-0.0833*m_f - 0.4624*m_t)
    M45 = l_t*(0.0833*l_t*m_f + 0.68*m_t*((l_f*ca.sin(q[1]+q[4]) + l_f*ca.sin(q[0]+q[4]-pi) + 0.68*l_t*ca.sin(q[1]-q[3]+q[4]) + l_t*ca.sin(q[0]-q[2]+q[4]-pi))*ca.sin(q[1]-q[3]+q[4]) + (l_f*ca.cos(q[1]+q[4]) + l_f*ca.cos(q[0]+q[4]-pi) + 0.68*l_t*ca.cos(q[1]-q[3]+q[4]) + l_t*ca.cos(q[0]-q[2]+q[4]-pi))*ca.cos(q[1]-q[3]+q[4])))

    M51 = -0.083*(l_f**2)*m_f - 0.545*(l_f**2)*m_f - 0.5*m_f*(0.702*(l_f**2) + 2.37*l_f*l_t*ca.cos(q[2]) + 2*(l_t**2)) - 0.5*m_f*(0*(l_f**2)*ca.sin(q[0]-q[1]) - 1.185*(l_f**2)*ca.cos(q[0]-q[1]) + 2*(l_f**2) - 0*l_f*l_t*ca.sin(-q[0]+q[1]+q[2]) - 1.185*l_f*l_t*ca.cos(-q[0]+q[1]+q[2]) + 4*l_f*l_t*ca.cos(q[2]) + 2*(l_t**2)) - 0.5*m_t*((l_f*ca.sin(q[0]+q[4]-pi) + l_t*ca.sin(q[0]-q[2]+q[4]-pi)) * (2*l_f*ca.sin(q[1]+q[4]) + 2*l_f*ca.sin(q[0]+q[4]-pi) + 1.36*l_t*ca.sin(q[1]-q[3]+q[4]) + 2*l_t*ca.sin(q[0]-q[2]+q[4]-pi)) + (l_f*ca.cos(q[0]+q[4]-pi) + l_t*ca.cos(q[0]-q[2]+q[4]-pi)) * (2*l_f*ca.cos(q[1]+q[4]) + 2*l_f*ca.cos(q[0]+q[4]-pi) + 1.36*l_t*ca.cos(q[1]-q[3]+q[4]) + 2*l_t*ca.cos(q[0]-q[2]+q[4]-pi))) - 10*(l_f*ca.sin(q[0]+q[4]-pi) + l_t*ca.sin(q[0]-q[2]+q[4]-pi)) * (2*l_f*ca.sin(q[0]+q[4]-pi) + 2*l_t*ca.sin(q[0]-q[2]+q[4]-pi) + 0.85*ca.sin(q[4])) - 10*(l_f*ca.cos(q[0]+q[4]-pi) + l_t*ca.cos(q[0]-q[2]+q[4]-pi)) * (2*l_f*ca.cos(q[0]+q[4]-pi) + 2*l_t*ca.cos(q[0]-q[2]+q[4]-pi) + 0.85*ca.cos(q[4]))
    M52 = -0.083*(l_f**2)*m_f + 0.29625*l_f*m_f*(0*l_f*ca.sin(q[0]-q[1]) + 2*l_f*ca.cos(q[0]-q[1]) - 1.185*l_f + 0*l_t*ca.sin(-q[0]+q[1]+q[2]) + 2*l_t*ca.cos(-q[0]+q[1]+q[2])) - 0.0833*(l_t**2)*m_f - 0.5*m_t*((l_f*ca.sin(q[1]+q[4]) + 0.68*l_t*ca.sin(q[1]-q[3]+q[4])) * (2*l_f*ca.sin(q[1]+q[4]) + 2*l_f*ca.sin(q[0]+q[4]-pi) + 1.36*l_t*ca.sin(q[1]-q[3]+q[4]) + 2*l_t*ca.sin(q[0]-q[2]+q[4]-pi)) + (l_f*ca.cos(q[1]+q[4]) + 0.68*l_t*ca.cos(q[1]-q[3]+q[4])) * (2*l_f*ca.cos(q[1]+q[4]) + 2*l_f*ca.cos(q[0]+q[4]-pi) + 1.36*l_t*ca.cos(q[1]-q[3]+q[4]) + 2*l_t*ca.cos(q[0]-q[2]+q[4]-pi)))
    M53 = l_t*(0*l_f*m_f*ca.sin(-q[0]+q[1]+q[2]) - 0.5925*l_f*m_f*ca.cos(-q[0]+q[1]+q[2]) - 0.5925*l_f*m_f*ca.cos(-q[0]+q[1]+q[2]) + 1.5925*l_f*m_f*cos(q[2]) - 0*l_f*m_t*ca.sin(-q[0]+q[1]+q[2]) - l_f*m_f*ca.cos(-q[0]+q[1]+q[2]) + l_f*m_t*ca.cos(q[2]) + 20*l_f*ca.cos(q[2]) + 2*l_t*m_f - 0.68*l_t*m_t*ca.sin(q[0]-q[2])*ca.sin(q[1])*ca.cos(q[3]) - 0.68*l_t*m_t*ca.sin(q[0]+q[3])*ca.sin(q[2]) + 0*l_t*m_t*ca.sin(q[0]-q[1]-q[2]+q[3]) + 0.68*l_t*m_t*ca.sin(q[0])*ca.sin(q[3])*ca.cos(q[1]+q[2]) - 0.68*l_t*m_t*ca.cos(q[1]-q[3])*ca.cos(q[0])*ca.cos(q[2]) + 1.545*l_t*m_t + 20*l_t + -8.5*ca.cos(q[0]-q[2]))
    M54 = l_t*(0.0833*l_t*m_f + 0.34*m_t*((2*l_f*ca.sin(q[1]+q[4]) + 2*l_f*ca.sin(q[0]+q[4]-pi) + 1.36*l_t*ca.sin(q[1]-q[3]+q[4]) + 2*l_t*ca.sin(q[0]-q[2]+q[4]-pi))*ca.sin(q[1]-q[3]+q[4]) + (2*l_f*ca.cos(q[1]+q[4]) + 2*l_f*ca.cos(q[0]+q[4]-pi) + 1.36*l_t*ca.cos(q[1]-q[3]+q[4]) + 2*l_t*ca.cos(q[0]-q[2]+q[4]-pi))*ca.cos(q[1]-q[3]+q[4])))
    M55 = -0*(l_f**2)*m_f*ca.sin(q[0]-q[1]) - 1.868*(l_f**2)*m_f + 0*l_f*l_t*m_f*ca.sin(-q[0]+q[1]+q[2]) + 1.185*l_f*l_t*m_f*ca.cos(-q[0]+q[1]+q[2]) - 2.0833*(l_t**2)*m_f - 0.545*(l_t**2)*m_t - m_t*((0*l_f*ca.sin(q[0]+q[4]) - l_f*ca.cos(q[0]+q[4]) + l_f*ca.cos(q[1]+q[4]) + 0*l_t*ca.sin(q[0]-q[2]+q[4]) - l_t*ca.cos(q[0]-q[2]+q[4]) + 0.68*l_t*ca.cos(q[1]-q[3]+q[4]))**2) - 20*((0*l_f*ca.sin(q[0]+q[4]) - l_f*ca.cos(q[0]+q[4]) + 0*l_t*ca.sin(q[0]-q[2]+q[4]) - l_t*ca.cos(q[0]-q[2]+q[4]) + 0.425*ca.cos(q[4]))**2) - 20*((l_f*ca.sin(q[0]+q[4]) + 0*l_f*ca.cos(q[0]+q[4]) + l_t*ca.sin(q[0]-q[3]+q[4]) + 0*l_t*ca.cos(q[0]-q[2]+q[4]) - 0.425*ca.sin(q[4]))**2) - 2.22

    M1 = ca.horzcat(M11, M12, M13, M14, M15)
    M2 = ca.horzcat(M21, M22, M23, M24, M25)
    M3 = ca.horzcat(M31, M32, M33, M34, M35)
    M4 = ca.horzcat(M41, M42, M43, M44, M45)
    M5 = ca.horzcat(M51, M52, M53, M54, M55)

    M = ca.vertcat(M1, M2, M3, M4, M5)

    return M

M_m = M(q, m_f, m_t, l_f, l_t)

Vector matrix: obtained by substituting mode shape terms with null value

In [10]:
def C(q, dq, mass, length_total, length_r, g):
    L1, L2, L3, L4, L5 = length_total[0], length_total[1], length_total[2], length_total[3], length_total[4]
    m1, m2, m3, m4, m5 = mass[0], mass[1], mass[2], mass[3], mass[4]
    r1, r2, r3, r4, r5 = length_r[0], length_r[1], length_r[2], length_r[3], length_r[4]
    q1, q2, q3, q4, q5 = q[0], q[1], q[2], q[3], q[4]
    dq1, dq2, dq3, dq4, dq5 = dq[0], dq[1], dq[2], dq[3], dq[4]    

    #Define each element of the C
    C1 = (((0.5000000000*m3 - 0.5000000000*m2)*dq5**2 + ((-1.000000000*m2 + 1.000000000*m3)*dq3 + 1.000000000*dq1*m3 - 1.000000000*dq1*m2)*dq5 + (0.5000000000*m3 - 0.5000000000*m2)*dq3**2 + dq1*(-1.000000000*m2 + 1.000000000*m3)*dq3 + 0.5000000000*dq1**2*m3 - 0.5000000000*dq1**2*m2)*L3**2 + r3*((-1.000000000*m3 + 1.000000000*m2)*dq5**2 + ((-2.000000000*m3 + 2.000000000*m2)*dq3 - 2.000000000*dq1*m3 + 2.000000000*dq1*m2)*dq5 + (-1.000000000*m3 + 1.000000000*m2)*dq3**2 + dq1*(-2.000000000*m3 + 2.000000000*m2)*dq3 - 1.000000000*dq1**2*m3 + 1.000000000*dq1**2*m2)*L3 + r3**2*(0.5000000000*m3 - 0.5000000000*m2)*dq5**2 + r3**2*((-1.000000000*m2 + 1.000000000*m3)*dq3 + 1.000000000*dq1*m3 - 1.000000000*dq1*m2)*dq5 + r3**2*(0.5000000000*m3 - 0.5000000000*m2)*dq3**2 + dq1*r3**2*(-1.000000000*m2 + 1.000000000*m3)*dq3 + 0.5000000000*dq1**2*m3*r3**2 - 0.5000000000*dq1**2*m2*r3**2)*ca.sin(2*q5 + 2*q1 + 2*q3) + m4*r4*(-1.000000000*dq5**2 - 2.000000000*(1.000000000*dq2 + 1.000000000*dq4)*dq5 - 2.000000000*dq2*dq4 - 1.000000000*dq2**2 - 1.000000000*dq4**2)*L3*ca.sin(q1 + q3 - q2 - q4) + L1*m4*r4*(-1.000000000*dq5**2 - 2.000000000*(1.000000000*dq2 + 1.000000000*dq4)*dq5 - 2.000000000*dq2*dq4 - 1.000000000*dq2**2 - 1.000000000*dq4**2)*ca.sin(q1 - q2 - q4) + ((-1.000000000*L2*m4 - 1.000000000*m2*r2)*dq5**2 - 2.000000000*dq2*(1.000000000*L2*m4 + 1.000000000*m2*r2)*dq5 - 1.000000000*L2*dq2**2*m4 - 1.000000000*dq2**2*m2*r2)*L3*ca.sin(q1 + q3 - q2) + L1*((-1.000000000*L2*m4 - 1.000000000*m2*r2)*dq5**2 - 2.000000000*dq2*(1.000000000*L2*m4 + 1.000000000*m2*r2)*dq5 - 1.000000000*dq2**2*(1.000000000*L2*m4 + 1.000000000*m2*r2))*ca.sin(q1 - q2) + g*((m3 + m2 + m4 + m5 + m1)*L3 - 1.*r3*m3)*ca.sin(q5 + q1 + q3) + g*((m1 + m2 + m4 + m5)*L1 - 1.*r1*m1)*ca.sin(q5 + q1) - 1.000000000*L3*ca.sin(q1 + q3)*dq5**2*m5*r5 + (((-2.000000000*m5 - 2.000000000*m4 - 2.000000000*m2 - 2.000000000*m1)*L1 + 2.000000000*r1*m1)*dq3*dq5 + ((-1.000000000*m5 - 1.000000000*m4 - 1.000000000*m2 - 1.000000000*m1)*L1 + 1.000000000*r1*m1)*dq3**2 + dq1*((-2.000000000*m5 - 2.000000000*m4 - 2.000000000*m2 - 2.000000000*m1)*L1 + 2.000000000*r1*m1)*dq3)*L3*ca.sin(q3) - 1.000000000*L1*ca.sin(q1)*dq5**2*m5*r5
    C2 = -1.0*m2*(-L3*(dq5 + dq1 + dq3)**2*ca.sin(q5 + q1 + q3) - L1*(dq5 + dq1)**2*ca.sin(q5 + q1) + r2*(dq5 + dq2)**2*ca.sin(q5 + q2))*r2*ca.cos(q5 + q2) - 1.0*m2*(L3*(dq5 + dq1 + dq3)**2*ca.cos(q5 + q1 + q3) + L1*(dq5 + dq1)**2*ca.cos(q5 + q1) - r2*(dq5 + dq2)**2*ca.cos(q5 + q2))*r2*ca.sin(q5 + q2) + ((-1.*dq5**2*r4 + (-2.*dq2 - 2.*dq4)*r4*dq5 + (-1.*dq2**2 - 2.*dq2*dq4 - 1.*dq4**2)*r4)*ca.sin(q5 + q2 + q4) + (dq5**2*L3 + (2.*L3*dq1 + 2.*dq3*L3)*dq5 + L3*dq1**2 + 2.*L3*dq1*dq3 + dq3**2*L3)*ca.sin(q5 + q1 + q3) + (L1*dq1**2 + 2.*L1*dq1*dq5 + L1*dq5**2)*ca.sin(q5 + q1) + (-1.*L2*dq2**2 - 2.*L2*dq2*dq5 - 1.*L2*dq5**2)*ca.sin(q5 + q2))*m4*(L2*ca.cos(q5 + q2) + r4*ca.cos(q5 + q2 + q4)) - (L2*ca.sin(q5 + q2) + r4*ca.sin(q5 + q2 + q4))*m4*((-1.*dq5**2*r4 + (-2.*dq2 - 2.*dq4)*r4*dq5 + (-1.*dq2**2 - 2.*dq2*dq4 - 1.*dq4**2)*r4)*ca.cos(q5 + q2 + q4) + (dq5**2*L3 + (2.*L3*dq1 + 2.*dq3*L3)*dq5 + L3*dq1**2 + 2.*L3*dq1*dq3 + dq3**2*L3)*ca.cos(q5 + q1 + q3) + (L1*dq1**2 + 2.*L1*dq1*dq5 + L1*dq5**2)*cos(q5 + q1) + (-1.*L2*dq2**2 - 2.*L2*dq2*dq5 - 1.*L2*dq5**2)*ca.cos(q5 + q2)) - m2*g*r2*ca.sin(q5 + q2) - m4*g*(L2*ca.sin(q5 + q2) + r4*ca.sin(q5 + q2 + q4))
    C3 = (((0.5000000000*m3 - 0.5000000000*m2)*dq5**2 + (dq1*(-1.000000000*m2 + 1.000000000*m3) + 1.000000000*dq3*m3 - 1.000000000*dq3*m2)*dq5 + (0.5000000000*m3 - 0.5000000000*m2)*dq1**2 + dq1*(-1.000000000*m2 + 1.000000000*m3)*dq3 + 0.5000000000*dq3**2*m3 - 0.5000000000*dq3**2*m2)*L3**2 + r3*((-1.000000000*m3 + 1.000000000*m2)*dq5**2 + (dq1*(-2.000000000*m3 + 2.000000000*m2) - 2.000000000*dq3*m3 + 2.000000000*dq3*m2)*dq5 + (-1.000000000*m3 + 1.000000000*m2)*dq1**2 + dq1*(-2.000000000*m3 + 2.000000000*m2)*dq3 - 1.000000000*dq3**2*m3 + 1.000000000*dq3**2*m2)*L3 + r3**2*(0.5000000000*m3 - 0.5000000000*m2)*dq5**2 + r3**2*(dq1*(-1.000000000*m2 + 1.000000000*m3) + 1.000000000*dq3*m3 - 1.000000000*dq3*m2)*dq5 + r3**2*(0.5000000000*m3 - 0.5000000000*m2)*dq1**2 + dq1*r3**2*(-1.000000000*m2 + 1.000000000*m3)*dq3 + 0.5000000000*dq3**2*m3*r3**2 - 0.5000000000*dq3**2*m2*r3**2)*ca.sin(2*q5 + 2*q1 + 2*q3) + m4*r4*(-1.000000000*dq5**2 - 2.000000000*(1.000000000*dq2 + 1.000000000*dq4)*dq5 - 2.000000000*dq2*dq4 - 1.000000000*dq2**2 - 1.000000000*dq4**2)*L3*ca.sin(q1 + q3 - q2 - q4) + ((-1.000000000*L2*m4 - 1.000000000*m2*r2)*dq5**2 - 2.000000000*dq2*(1.000000000*L2*m4 + 1.000000000*m2*r2)*dq5 - 1.000000000*L2*dq2**2*m4 - 1.000000000*dq2**2*m2*r2)*L3*ca.sin(q1 + q3 - q2) + g*((m3 + m2 + m4 + m5 + m1)*L3 - 1.*r3*m3)*ca.sin(q5 + q1 + q3) - 1.000000000*L3*ca.sin(q1 + q3)*dq5**2*m5*r5 + ((1.000000000*L1*m2 + 1.000000000*L1*m4 + (1.000000000*m1 + 1.000000000*m5)*L1 - 1.000000000*r1*m1)*dq5**2 + (2.000000000*L1*m2 + 2.000000000*L1*m4 + (2.000000000*m5 + 2.000000000*m1)*L1 - 2.000000000*r1*m1)*dq1*dq5 + (1.000000000*L1*m2 + 1.000000000*L1*m4 + (1.000000000*m1 + 1.000000000*m5)*L1 - 1.000000000*r1*m1)*dq1**2)*L3*ca.sin(q3)
    C4 = m4*r4*(-1.0*(-L3*(dq5 + dq1 + dq3)**2*ca.sin(q5 + q1 + q3) - L1*(dq5 + dq1)**2*ca.sin(q5 + q1) + L2*(dq5 + dq2)**2*ca.sin(q5 + q2) + r4*(dq5 + dq2 + dq4)**2*ca.sin(q5 + q2 + q4))*ca.cos(q5 + q2 + q4) - 1.0*(L3*(dq5 + dq1 + dq3)**2*ca.cos(q5 + q1 + q3) + L1*(dq5 + dq1)**2*ca.cos(q5 + q1) - L2*(dq5 + dq2)**2*ca.cos(q5 + q2) - r4*(dq5 + dq2 + dq4)**2*ca.cos(q5 + q2 + q4))*ca.sin(q5 + q2 + q4) - g*ca.sin(q5 + q2 + q4))
    C5 = (((0.5000000000*m3 - 0.5000000000*m2)*dq3**2 + (-1.000000000*m2 + 1.000000000*m3)*(dq5 + dq1)*dq3 + (0.5000000000*m3 - 0.5000000000*m2)*dq5**2 + dq1*(-1.000000000*m2 + 1.000000000*m3)*dq5 + (0.5000000000*m3 - 0.5000000000*m2)*dq1**2)*L3**2 + r3*((-1.000000000*m3 + 1.000000000*m2)*dq3**2 + (-2.000000000*m3 + 2.000000000*m2)*(dq5 + dq1)*dq3 + (-1.000000000*m3 + 1.000000000*m2)*dq5**2 + (-2.000000000*m3 + 2.000000000*m2)*dq1*dq5 + (-1.000000000*m3 + 1.000000000*m2)*dq1**2)*L3 + r3**2*(0.5000000000*m3 - 0.5000000000*m2)*dq3**2 + r3**2*(-1.000000000*m2 + 1.000000000*m3)*(dq5 + dq1)*dq3 + r3**2*(0.5000000000*m3 - 0.5000000000*m2)*dq5**2 + r3**2*(-1.000000000*m2 + 1.000000000*m3)*dq1*dq5 + r3**2*(0.5000000000*m3 - 0.5000000000*m2)*dq1**2)*ca.sin(2*q5 + 2*q1 + 2*q3) + m4*r4*(1.000000000*dq3**2 + 2.000000000*(1.000000000*dq1 + 1.000000000*dq5)*dq3 + (2.000000000*dq1 - 2.000000000*dq2 - 2.000000000*dq4)*dq5 + 1.000000000*dq1**2 - 2.000000000*dq2*dq4 - 1.000000000*dq2**2 - 1.000000000*dq4**2)*L3*ca.sin(q1 + q3 - q2 - q4) + L1*m4*r4*((2.000000000*dq1 - 2.000000000*dq2 - 2.000000000*dq4)*dq5 + 1.000000000*dq1**2 - 2.000000000*dq2*dq4 - 1.000000000*dq2**2 - 1.000000000*dq4**2)*ca.sin(q1 - q2 - q4) + ((1.000000000*L2*m4 + 1.000000000*m2*r2)*dq3**2 + (2.000000000*L2*m4 + 2.000000000*m2*r2)*(dq5 + dq1)*dq3 + ((2.000000000*L2*m4 + 2.000000000*m2*r2)*dq1 - 2.000000000*L2*dq2*m4 - 2.000000000*dq2*m2*r2)*dq5 + (1.000000000*L2*m4 + 1.000000000*m2*r2)*dq1**2 - 1.000000000*L2*dq2**2*m4 - 1.000000000*dq2**2*m2*r2)*L3*ca.sin(q1 + q3 - q2) + L1*((2.000000000*(1.000000000*L2*m4 + 1.000000000*m2*r2)*dq1 - 2.000000000*dq2*m2*r2 - 2.000000000*L2*dq2*m4)*dq5 + 1.000000000*(1.000000000*L2*m4 + 1.000000000*m2*r2)*dq1**2 - 1.000000000*dq2**2*m2*r2 - 1.000000000*L2*dq2**2*m4)*ca.sin(q1 - q2) + g*((m3 + m2 + m4 + m5 + m1)*L3 - 1.*r3*m3)*ca.sin(q5 + q1 + q3) - 1.*m4*g*r4*ca.sin(q5 + q2 + q4) + m5*r5*(1.000000000*dq3**2 + 2.000000000*(1.000000000*dq1 + 1.000000000*dq5)*dq3 + 1.000000000*dq1**2 + 2.000000000*dq1*dq5)*L3*ca.sin(q1 + q3) + g*(L1*m4 + L1*m2 + (m1 + m5)*L1 - 1.*r1*m1)*ca.sin(q5 + q1) - 1.*g*(1.000000000*L2*m4 + 1.000000000*m2*r2)*ca.sin(q5 + q2) + ((-1.000000000*L1*m4 - 1.000000000*L1*m2 + (-1.000000000*m1 - 1.000000000*m5)*L1 + 1.000000000*r1*m1)*dq3**2 + (-2.000000000*L1*m4 - 2.000000000*L1*m2 + (-2.000000000*m5 - 2.000000000*m1)*L1 + 2.000000000*r1*m1)*(dq5 + dq1)*dq3)*L3*ca.sin(q3) + r4*L2*m4*(-2.000000000*dq5*dq4 - 1.000000000*dq4**2 - 2.000000000*dq2*dq4)*ca.sin(q4) + L1*m5*r5*(1.000000000*dq1**2 + 2.000000000*dq1*dq5)*ca.sin(q1) - 1.*ca.sin(q5)*g*m5*r5

    #Generate entire C matrix
    C_matrix = ca.vertcat(C1, C2, C3, C4, C5)
    return C_matrix

C_m = C(q, dq, mass, length, l_r, g)

In [11]:
# Array of controlled parameters u
def ControlledInputs(u):
    # Array B shows what inputs are active (from u1 to u4)
    B = np.array([[1, 0, 0, 0, 0],
                [0, 1, 0, 0, 0],
                [0, 0, 1, 0, 0],
                [0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0],
                ])
    B_u = B@u       # Set u5 to zero, meaning that head movement is not controlled
    return B_u

ControlledInputs(u)

MX(mac(
[[1, 0, 0, 0, 0], 
 [0, 1, 0, 0, 0], 
 [0, 0, 1, 0, 0], 
 [0, 0, 0, 1, 0], 
 [0, 0, 0, 0, 0]],u,zeros(5x1)))

Final equation of motion: M*(ei) + C - B*u = 0

In [12]:
# Final set of equations of motion
def e_motion_u(C, M, u):
    ei = ca.MX.sym('ei',5,1)                        # Declare mode shapes (ei)
    M_ei = M@ei                                     # M*ei
    e_motion_u = C + M_ei - ControlledInputs(u)     # Final equation of motion: C + M*ei - B*u = 0
    return e_motion_u

e_motion_u(C_m, M_m, u)

MX(@1=dq[4], @2=dq[2], @3=dq[0], @4=q[4], @5=q[0], @6=q[2], @7=dq[1], @8=dq[3], @9=q[1], @10=q[3], ((vertcat(((((((((((((((((((0.16*((((((-1.8*sq(@1))+((((-3.6*@2)+(3.2*@3))-(6.8*@3))*@1))+(-1.8*sq(@2)))+((-3.6*@3)*@2))+(3.2*(0.5*sq(@3))))-(6.8*(0.5*sq(@3)))))+(0.4*(0.128*((((((3.6*sq(@1))+((((7.2*@2)-(3.2*(2.*@3)))+(6.8*(2.*@3)))*@1))+(3.6*sq(@2)))+((7.2*@3)*@2))-(3.2*sq(@3)))+(6.8*sq(@3))))))+(-0.0294912*sq(@1)))+((0.016384*(((-3.6*@2)+(3.2*@3))-(6.8*@3)))*@1))+(-0.0294912*sq(@2)))+((-3.6*(0.016384*@3))*@2))+(0.016384*(3.2*(0.5*sq(@3)))))-(0.016384*(6.8*(0.5*sq(@3)))))*sin((((2.*@4)+(2.*@5))+(2.*@6))))+((0.4*(0.4096*(((((-sq(@1))-((2.*(@7+@8))*@1))-((2.*@7)*@8))-sq(@7))-sq(@8))))*sin((((@5+@6)-@9)-@10))))+((0.16384*(((((-sq(@1))-((2.*(@7+@8))*@1))-((2.*@7)*@8))-sq(@7))-sq(@8)))*sin(((@5-@9)-@10))))+((0.4*((((-2.3884*sq(@1))-((2.3884*(2.*@7))*@1))-(3.2*(0.4*sq(@7))))-(0.163*(6.8*sq(@7)))))*sin(((@5+@6)-@9))))+((0.4*(((-2.3884*sq(@1))-((2.3884*(2.*@7))*@1))-(2.3884*sq(@7))))*sin((@5-@9

## Solver
### State space calculation

In [13]:
def d_state(q, dq, u, M, C):
    B_u = ControlledInputs(u)          # Calculate acting inputs and the inverse of the mass matrix
    M_inverse = ca.inv(M)              # Inverse of matrix M
    ddq_u = M_inverse@(B_u - C)        # Acceleration term given in the assigment notes (comes from solving for ei the equations of motion)
    
    dq_x = ca.horzcat(dq)              # Generate array of speed
    ddq_x = ca.horzcat(ddq_u)          # Generate array of acceleration
    dx = ca.vertcat(dq_x, ddq_x)       # State-space model
    #print(dx.shape)

    return dx

d_state(q, dq, u, M_m, C_m)

MX(@1=dq[4], @2=dq[2], @3=dq[0], @4=q[4], @5=q[0], @6=q[2], @7=dq[1], @8=dq[3], @9=q[1], @10=q[3], vertcat(dq, mac((horzcat(horzcat(((((((((((-1.4343*sq(l_f))*m_f)-(sq(l_f)*m_t))-(20*sq(l_f)))-(((3.185*l_f)*m_f)*cos(q[2])))-(((2.*l_f)*m_t)*cos(q[2])))-(((40*l_t)*m_t)*cos(q[2])))-((2.*sq(l_t))*m_f))-((1.545*sq(l_t))*m_t))-(20*sq(l_t))), (-(m_t*(((l_f*sin(((q[1]-q[3])+q[4])))*((l_f*sin(((q[0]+q[4])-3.14159)))+(l_t*sin((((q[0]-q[2])+q[4])-3.14159)))))+(((l_f*cos((q[1]+q[4])))+((0.68*l_t)*cos(((q[1]-q[3])+q[4]))))*((l_f*cos(((q[0]+q[4])-3.14159)))+(l_t*cos((((q[0]-q[2])+q[4])-3.14159)))))))), (l_t*((((((((1.5925*l_f)*m_f)*cos(q[2]))+((l_f*m_f)*cos(q[2])))+((20*l_f)*cos(q[2])))+((2.*l_t)*m_f))+((1.5457*l_t)*m_t))+(20*l_t))), (((0.68*l_t)*m_t)*((((l_f*sin(((q[0]+q[4])-3.14159)))+(l_t*sin((((q[0]-q[2])+q[4])-3.14159))))*sin(((q[1]-q[3])+q[4])))+(((l_f*cos(((q[0]+q[4])-3.14159)))+(l_t*cos((((q[0]-q[2])+q[4])-3.14159))))*cos(((q[1]-q[2])+q[4]))))), (((((((((-0.0833*sq(l_f))*m_f)-((0.545*sq(l_t)

### Openloop function
Substitute symbolical values with numerical values and calculate state-space model

In [14]:
# Openloop function to analyse each system each sample period
def openloop(x, u, mass, length, l_r, g):
    q, dq = x[:5], x[5:]                        # First set desired state
    m_f = mass[0]                               # Mass of femur
    m_t = mass[2]                               # Mass of tibia
    l_f = length[0]                             # Length of femur
    l_t = length[2]                             # Length of tibia

    M_matr = M(q, m_f, m_t, l_f, l_t)           # Calculate mass matrix
    C_matr = C(q, dq, mass, length, l_r, g)     # Calculate vector matrix
    
    dx = d_state(q, dq, u, M_matr, C_matr)      # State-space model
    return dx

## Animation function

In [15]:
# Animated pendulum
def animate_pendulum(t_ar, pos_x, pos_y, phi_sol, lengh, filename="GIF_try3.gif", slowdown=1, skip_frames=1):
    # Set up the figure, the axis, and the plot elements
    fig, (ax1, ax2) = plt.subplots(1, 2) # Ax1 human, Ax2 angles
    plt.subplots_adjust(wspace=0.5)  # Adjust the width spacing
    fig.suptitle('Biped walker gravity colapse')
    ax1.set_title('Biped walker movement')
    ax1.set_xlim(- 1.475, 3) # Full body aligned + 0.05
    ax1.set_ylim(- 1.475, 1.475) # Full body aligned + 0.05
    ax1.set_aspect(1)
    ax2.set_title('Joint Angles')
    y2min = np.min(phi_sol)
    y2max = np.max(phi_sol)   
    # y2min = -5*np.pi
    # y2max = 5*np.pi
    ax2.set_ylim(y2min-0.5, y2max+0.5) 
    ax2.set_xlim(0, 5) # t[0], t[-1]) 
    ax2.set_xlabel('Time [s]')
    ax2.set_ylabel('Angle [rad]')
    # ax2.set_aspect((t[-1]-t[0])/(y2max-y2min))
    def radians_formatter(x, pos):
        return "{:.0f}".format(x/(np.pi))+"pi"
    #plt.gca().yaxis.set_major_formatter(FuncFormatter(radians_formatter))
    # display time
    time_text = ax1.text(0.04, 0.9, '', transform=ax1.transAxes)

    # create a rectangular Floor
    Floor_width = 10*1.475 # All the image
    Floor_height = 0.05
    rect = Rectangle([0 , -Floor_height],
                     Floor_width, Floor_height,
                     fill=True, color='red', ec='black')
    ax1.add_patch(rect)

    # Drawing the pendulum and lines
    Biped_walker, = ax1.plot([], [], lw=2, marker='o', markersize=6)
    Biped_walker_leg2, = ax1.plot([], [], lw=2, marker='o', markersize=6, color='red')
    phi1line, = ax2.plot([], [])
    phi2line, = ax2.plot([], [])
    phi3line, = ax2.plot([], [])
    phi4line, = ax2.plot([], [])
    phi5line, = ax2.plot([], [])

    # initialization function: plot the background of each frame
    def init():
        time_text.set_text('')
        rect.set_xy((0- Floor_width / 2.0,
                     -Floor_height))
        Biped_walker.set_data([], [])
        Biped_walker_leg2.set_data([], [])
        phi1line.set_data([], [])
        phi2line.set_data([], [])
        phi3line.set_data([], [])
        phi4line.set_data([], [])
        phi5line.set_data([], [])
        return time_text, rect, Biped_walker,Biped_walker_leg2, phi1line, phi2line, phi3line, phi4line, phi5line

    # animation function: update the objects
    def animate(i):
        new_i = i 
        time_text.set_text('time = {:2.2f}'.format(t_ar[i]))
        rect.set_xy((0 - Floor_width / 2.0, -Floor_height))
        x = pos_x[i]
        y = pos_y[i]
        # Set the walker points whell organized
        Biped_walker.set_data(pos_x[i], pos_y[i])
        Biped_walker_leg2.set_data(x[4:7], y[4:7])
        # Set each angle position
        phi1line.set_data(t_ar[:i], phi_sol[:i, 0])
        phi2line.set_data(t_ar[:i], phi_sol[:i, 1])
        phi3line.set_data(t_ar[:i], phi_sol[:i, 2])
        phi4line.set_data(t_ar[:i], phi_sol[:i, 3])
        phi5line.set_data(t_ar[:i], phi_sol[:i, 4])
        ax2.legend(['phi1', 'phi2', 'phi3', 'phi4', 'phi5'])
        return time_text, rect, Biped_walker,Biped_walker_leg2, phi1line, phi2line, phi3line, phi4line, phi5line

    # call the animator function
    anim = animation.FuncAnimation(fig, animate, frames=len(t_ar),
                                   init_func=init,
                                   interval=t_ar[-1] / len(t_ar) * 1000 * slowdown,
                                   blit=False, repeat=False)
    plt.show(block=True)

    # save the animation if a filename is given
    if filename is not None:
        plt.rcParams['animation.ffmpeg_path'] = '/path/to/ffmpeg'
        anim.save(filename, fps=30)

## Heel Strike Impact Model

In [16]:
def change_of_leg(q_simulated, dq_simulated, M_matrix, C_matrix, joint):
    # Define leg position right before impact moment
    q1_plus, q2_plus, q3_plus, q4_plus, q5_plus = dynamicsymbols('q1_plus q2_plus q3_plus q4_plus q5_plus')
    qminus_symb = [q1_plus, q2_plus, q3_plus, q4_plus, q5_plus]
    r_minus = np.hstack([q_simulated[-1], dq_simulated[-1]])
    q_minus = r_minus[:5]
    dq_minus = r_minus[5:]
    M_minus_sym = M_matrix
    C_minus_sym = C_matrix
    joint_minus_sym = joint

    # Leg change matrix
    leg_change = np.array([[0, 1, 0, 0, 0],
                        [1, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0],
                        [0, 0, 1, 0, 0],
                        [0, 0, 0, 0, 1]])

    # Calculate q for after impact
    q_plus_t = np.dot(leg_change, q)
    q_plus_sym = np.dot(leg_change, qminus_symb)
    q_plus = np.dot(leg_change, q_minus)
    #print('q minus:' + f'{q_minus}')
    #print('q plus:' + f'{q_plus}')

    # Change symbolic mass matrix
    M_plus_sym = M_minus_sym.subs({q1: q_plus_sym[0], q2: q_plus_sym[1], q3: q_plus_sym[2], q4: q_plus_sym[3], q5: q_plus_sym[4]})
    M_plus_sym = M_plus_sym.subs({q_plus_sym[0]: q_plus_t[0], q_plus_sym[1]: q_plus_t[1], q_plus_sym[2]: q_plus_t[2], q_plus_sym[3]: q_plus_t[3], q_plus_sym[4]: q_plus_t[4]})
    #print('M-: ' + f'{M_minus_sym}')
    #print('M+: ' + f'{M_plus_sym}')

    # Change symbolic vector matrix
    C_plus_sym = C_minus_sym.subs({q1: q_plus_sym[0], q2: q_plus_sym[1], q3: q_plus_sym[2], q4: q_plus_sym[3], q5: q_plus_sym[4]})
    C_plus_sym = C_plus_sym.subs({q_plus_sym[0]: q_plus_t[0], q_plus_sym[1]: q_plus_t[1], q_plus_sym[2]: q_plus_t[2], q_plus_sym[3]: q_plus_t[3], q_plus_sym[4]: q_plus_t[4]})
    #print('C-: ' + f'{C_minus_sym}')
    #print('C+: ' + f'{C_plus_sym}')

    # Change joint_order matrix
    joint_plus_sym = sp.zeros(2, 7)
    for i in range(2):
        for j in range(6):
            joint_plus_sym[i, j+1] = joint_minus_sym[i, j+1].subs({q1: q_plus_sym[0], q2: q_plus_sym[1], q3: q_plus_sym[2], q4: q_plus_sym[3], q5: q_plus_sym[4]})
            joint_plus_sym[i, j+1] = joint_plus_sym[i, j+1].subs({q_plus_sym[0]: q_plus_t[0], q_plus_sym[1]: q_plus_t[1], q_plus_sym[2]: q_plus_t[2], q_plus_sym[3]: q_plus_t[3], q_plus_sym[4]: q_plus_t[4]})
    # print('Joint -: ' + f'{joint_minus_sym}')
    # print('Joint +: ' + f'{joint_plus_sym}')

    # Calculate mass matrix before and after impact
    M_minus = M_minus_sym.subs({q1: q_minus[0], q2: q_minus[1], q3: q_minus[2], q4: q_minus[3], q5: q_minus[4]})
    M_plus = M_minus_sym.subs({q1: q_plus[0], q2: q_plus[1], q3: q_plus[2], q4: q_plus[3], q5: q_plus[4]})
    M_plus = np.array(M_plus, dtype='float64')

    # Calculate speed for after impact: M+ * dq+ = M- * dq- --> dq+ = ((M+)^-1) * (M- * dq-)
    momentum_minus = np.dot(M_minus, dq_minus)
    M_plus_inv = np.linalg.inv(M_plus)
    dq_plus = np.dot(M_plus_inv, momentum_minus)

    #print('dq minus:' + f'{dq_minus}')
    #print('dq plus:' + f'{dq_plus}')

    return q_plus, dq_plus, M_plus_sym, C_plus_sym, joint_plus_sym

In [45]:
last_X = opti.debug.value(X)
print(last_X)

[[-7.04923569e-01 -7.22557346e-01 -2.15552993e+00 -2.18420008e+00
  -1.46797209e+00 -1.60559731e+00 -1.52739924e+00 -1.48385387e+00
  -1.35582061e+00 -1.33142325e+00 -1.28604836e+00 -1.24454484e+00
  -1.19617248e+00 -1.74725853e+00 -1.70823804e+00 -1.61856684e+00
  -1.37076301e+00 -1.33244687e+00 -1.33599752e+00 -1.34605178e+00
  -1.34575145e+00]
 [ 1.10858339e+00  1.04752046e+00  6.44704497e-01  6.49699708e-01
   6.91571886e-01  7.14318681e-01  6.94122673e-01  6.93997034e-01
   6.93912288e-01  6.95329439e-01  6.89632532e-01  6.79333298e-01
   6.66175028e-01  7.04251027e-01  7.35182662e-01  7.44616512e-01
   8.17237065e-01  8.24930556e-01  8.70873526e-01  9.83018870e-01
   1.67476323e+00]
 [ 4.49763417e-03 -7.00035626e-03 -3.34581225e-01 -2.70392845e-01
   3.70990577e-01  4.08434638e-01  8.45192944e-01  5.75672213e-01
   5.51835140e-01  5.16972821e-01  4.96400232e-01  3.63413227e-01
   1.97671898e-01 -5.77296038e-01 -1.00059654e-01 -2.85842060e-01
   4.86039007e-01  4.58596591e-01  4.0

## Constraint value problem

In [49]:
###### DATA #######
walking_speed = 1.12             # Average human step speed in [m/s]
freq_step = 0.92                 # Average human step frequency
L = (walking_speed*freq_step)/2  # Step size = (speed*frequency) / 2
#T = 1/freq_step                  # Average human step duration = 1/frequency
T = 0.5
N = 20                           # Number of control intervals
dt = T/N                         # Length of a control interval
ang_sp = 0                       # Starting speed of free leg

# Inverse kinematics equations for start and final angles
angle_1 = pi + math.asin((L / 2) / 0.8)
angle_2 = pi - math.asin((L / 2) / 0.8)

###### OPTIMIZATION PROBLEM ######
opti = casadi.Opti()             # Optimization problem

####### Decision variables #######
X = opti.variable(10, N+1)             # state trajectory
q_ca   = X[:5, :]                      # q
dq_ca = X[5:, :]                       # dq
pos_y_ca, pos_x_ca = sim_pos(q_ca)     # Absolute positions
U = opti.variable(5, N)                # control trajectory (input torque)
T = opti.variable()                    # final time
cost = 0                               # Cost initialization

# Loop over control intervals with Runge-Kutta 4 solver
for k in range(N):
   # Simulate possible responses
   k1 = openloop(X[:, k],              U[:, k], mass, length, l_r, 9.81)
   k2 = openloop(X[:, k] + dt*(k1/2),  U[:, k], mass, length, l_r, 9.81)
   k3 = openloop(X[:, k] + dt*(k2/2),  U[:, k], mass, length, l_r, 9.81)
   k4 = openloop(X[:, k] + dt*k3,      U[:, k], mass, length, l_r, 9.81)

   # Calculate weighted response and save values
   x_next = X[:, k] + dt*(k1 + 2*k2 + 2*k3 + k4)/6
   q_x_1, dq_x_1 = X[:5, k], X[5:, k]
   
   ######### Cost rate #########
   cost = cost + ((U[0, k]**2 + U[1, k]**2 + U[2, k]**2 + U[3, k]**2 + U[4, k]**2) / 2)

   opti.subject_to(X [:, k+1] == x_next)      # Close the gaps

# Initial state
q_ini = [angle_1, angle_2, 0, 0, 0]       # Initial angles
dq_ini = [0 ,0, 0, ang_sp, 0]             # Initial speed
x_ini_des = ca.horzcat(q_ini, dq_ini)     # Initial state

# Final state
q_end = [angle_2, angle_1, 0, 0, 0]       # Final angles
dq_end = [0, 0, 0, -ang_sp, 0]            # Final speed
x_end_des = ca.horzcat(q_end, dq_end)     # Final state

######### Boundary conditions #########
# Initial state
opti.subject_to(q_ca[:, 0] == q_ini)      
opti.subject_to(dq_ca[:, 0] == dq_ini)
#opti.subject_to(U[:, 0] == [0, 0, 0, 0, 0])
# Final state
opti.subject_to(q_ca[:, N] == q_end)      
opti.subject_to(dq_ca[:, N] == dq_end)
#opti.subject_to(U[:, N] == [0, 0, 0, 0, 0])

######### Constraints #########
opti.subject_to(vec(q_ca[2:3, :]) <= pi*(2/3))     # Rotation constraint: upper limit for tibia rotation wrt femur
opti.subject_to(vec(q_ca[2:3, :]) >= 0)            # Rotation constraint: lower limit for tibia rotation wrt femur
opti.subject_to(vec(q_ca[:1, :]) <= -pi)           # Rotation constraint: upper limit for femur rotation wrt trunk
opti.subject_to(vec(q_ca[:1, :]) >= -pi*(3/2))     # Rotation constraint: lower limit for femur rotation wrt trunk
opti.subject_to(vec(q_ca[4, :]) > -pi/4)           # Rotation constraint: upper limit for head swinging
opti.subject_to(vec(q_ca[4, :]) < pi/4)            # Rotation constraint: lower limit for head swinging
# opti.subject_to(vec(pos_y_ca[1:6, :]) > 0)         # Rotation constraint: lower limit for head swinging

opti.minimize(cost)

# Inital guess for the solver
opti.set_initial(U, 1)
opti.set_initial(X, last_X)

###### Solve the NLP using IPOPT #######
opti.solver('ipopt')                        # Set numerical backend
sol = opti.solve()                          # Actual solve

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     3220
Number of nonzeros in inequality constraint Jacobian.:      126
Number of nonzeros in Lagrangian Hessian.............:     2400

Total number of variables............................:      310
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      220
Total number of inequality constraints...............:      126
        inequality constraints with only lower bounds:       63
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       63

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  5.0000000e+01 1.63e+13 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:157:
.../casadi/core/optistack_internal.cpp:996: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Restoration_Failed'

In [50]:
opti.debug.show_infeasibilities()

Violated constraints (tol 0), in order of declaration:
------- i = 0/346 ------ 
0 <= -3.16736 <= 0 (viol 3.16736)
Opti constraint of shape 10x1, defined at C:\Users\uamen\AppData\Local\Temp\ipykernel_24268\100757980.py:42
At nonzero 0.
------- i = 1/346 ------ 
0 <= -5.5201e-05 <= 0 (viol 5.5201e-05)
Opti constraint of shape 10x1, defined at C:\Users\uamen\AppData\Local\Temp\ipykernel_24268\100757980.py:42
At nonzero 1.
------- i = 2/346 ------ 
0 <= -0.0390854 <= 0 (viol 0.0390854)
Opti constraint of shape 10x1, defined at C:\Users\uamen\AppData\Local\Temp\ipykernel_24268\100757980.py:42
At nonzero 2.
------- i = 3/346 ------ 
0 <= 0.0516621 <= 0 (viol 0.0516621)
Opti constraint of shape 10x1, defined at C:\Users\uamen\AppData\Local\Temp\ipykernel_24268\100757980.py:42
At nonzero 3.
------- i = 4/346 ------ 
0 <= 0.035736 <= 0 (viol 0.035736)
Opti constraint of shape 10x1, defined at C:\Users\uamen\AppData\Local\Temp\ipykernel_24268\100757980.py:42
At nonzero 4.
------- i = 5/346 ---

In [80]:
last_X = opti.debug.value(X)
#print(last_X)
q_sim_ca = last_X[:5]
#print(q_sim_ca[:, 0])
x_1, y_1 = sim_pos(q_sim_ca[:, 0])